## Levatamos Fermionic

In [149]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
        else:
            self.base, self.num_ele = basis, num_ele
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m_gen(basis, m):
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)
    
    it_set = np.arange(m_basis.size) #[::-1]
    for i, ii in tqdm(enumerate(it_set), total=m_basis.size):
        for j, jj in enumerate(it_set):
            # Generamos el operador
            op = m_basis.base[jj]*of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=basis.d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

# Levanta bases de QChem
def build_csv_basis(csvf, d):
    # Construimos la base
    ops = []
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split('  ')[1:]] # Operador en forma de lista
            op = of.FermionOperator(([(i, 1) for i in natop]))
            ops.append(op)
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**(d-1-i) for i in x])
            num_ele.append(natop_to_int(natop))
 
        # Determinamos m y d
        num = len(natop)
        assert d == m_level+1

    return fixed_basis(d, num = num, pairs = False, basis = ops, num_ele=num_ele)
        

In [150]:
basis = build_csv_basis('h4_square_h4_square_basis', 8)

rho_2_arrays = rho_m_gen(basis, 2)
# Indices
t_basis = fixed_basis(basis.d, num=2)
k_indices = get_kkbar_indices(t_basis)
print(k_indices)
# Arrays reducidos
rho_2_kkbar_arrays = rho_2_gen(basis, t_basis, idx_list = k_indices)


100%|██████████| 28/28 [00:14<00:00,  1.92it/s]


[0, 5, 14, 27]


100%|██████████| 4/4 [00:00<00:00, 14.04it/s]


### Determinación de estado

In [302]:
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.cluster.vq import kmeans, vq

A = np.array([1, 1, 2, 2.01])
e = 0.2

def sparsity_error(A, k=2):
    # L1
    sparsity_term = np.sum(np.abs(A))

    # Clustering 
    non_zero_coeffs = A[np.abs(A) > 1e-5]  # Sacamos los ceros
    if len(non_zero_coeffs) > 0:
        # Calculamos los clusters (k) y la distancia en ellos
        centroids, _ = kmeans(non_zero_coeffs, k)
        cluster_labels, distances = vq(non_zero_coeffs, centroids)
        cluster_error = np.sum(distances**2) 
    else:
        cluster_error = 0

    loss = 1 * sparsity_term + 2 * cluster_error
    return loss

sparsity_error(A, 2)

6.0100999999999996

In [303]:
# Input
rho_2_obj = np.sort(np.concatenate([np.repeat(1/12, 4+4*2), np.repeat(1/3, 2+2*2), np.repeat(3/4,4), np.repeat(0,6)]))

# Aux functions
r_eig = lambda x: np.sort(np.real(np.linalg.eigvals(x.todense())))
k = 2 # Número de clusters

# Loss definition
def loss_fun(vect):
    # Rho loss
    rho = rho_m(vect, rho_2_arrays)
    eigv = r_eig(rho)
    lrho = np.linalg.norm(rho_2_obj-eigv)
    # L1 + clustering loss
    l1 = sparsity_error(vect, k) 

    print(lrho, l1)
    return lrho+1/2*l1

opt = scipy.optimize.basinhopping(loss_fun, vect, niter=10)
print(opt) # 0.07 lrho
print(opt.x)
print(rho_2_obj)
print(clustering_error(opt.x, e=eps, debug=True))
r_eig(rho_m(opt.x, rho_2_arrays))

0.009498464459609678 3.221178952375899
0.009498454573820243 3.221178926936314
0.009498464459616461 3.2211789672770603
0.009498464459616506 3.2211789672770603
0.009498464459616275 3.2211789672758555
0.009498464459614522 3.2211789672765465
0.00949846450697295 3.2211789586753814
0.009498464459615639 3.2211789672765074
0.009498474345489212 3.221178977804502
0.009498464459616668 3.2211789672770603
0.009498464459616317 3.2211789672770603
0.009498464412296296 3.2211789460654345
0.009498464459614851 3.2211789672770603
0.009498464459616461 3.221178967275853
0.00949846445961684 3.221178967275705
0.009498464459632905 3.221178967276935
0.009498464364967088 3.2211789374967053
0.009498464459617025 3.2211789672758018
0.009498464459616671 3.221178967275939
0.009498464459617488 3.2211789672770603
0.009498464459616263 3.221178967275708
0.009498464364966687 3.2211789374967053
0.009498464459633302 3.221178967276936
0.009498464459617202 3.221178967275806
0.009498464459616917 3.2211789672770603
0.0094984644

array([2.66725179e-09, 3.48442024e-09, 4.55344480e-09, 5.61379968e-09,
       4.15878170e-07, 5.95007814e-07, 8.21235418e-02, 8.21235418e-02,
       8.21319987e-02, 8.21319987e-02, 8.21758102e-02, 8.21841540e-02,
       8.31358143e-02, 8.31358143e-02, 8.31405185e-02, 8.31405185e-02,
       8.31889161e-02, 8.31936181e-02, 3.34095414e-01, 3.34200616e-01,
       3.34200616e-01, 3.34243227e-01, 3.34348632e-01, 3.34348632e-01,
       7.48996198e-01, 7.49018701e-01, 7.49606868e-01, 7.49620555e-01])

In [304]:
for i, ii in enumerate(opt.x):
    if np.abs(ii) > 0.01:
        print(ii, basis.base[i])


print(1/np.sqrt(3), 1/np.sqrt(12))

-0.45379953648559135 1.0 [0^ 1^ 4^ 5^]
0.1442496882646369 1.0 [0^ 1^ 6^ 7^]
0.4537326343249632 1.0 [0^ 2^ 4^ 6^]
-0.14417924271650154 1.0 [0^ 2^ 5^ 7^]
-0.28833672943436967 1.0 [0^ 3^ 5^ 6^]
-0.28833701868700523 1.0 [1^ 2^ 4^ 7^]
-0.14417925927531242 1.0 [1^ 3^ 4^ 6^]
0.4109550267570592 1.0 [1^ 3^ 5^ 7^]
0.14424970123346037 1.0 [2^ 3^ 4^ 5^]
-0.4110014563804238 1.0 [2^ 3^ 6^ 7^]
0.5773502691896258 0.2886751345948129


In [259]:
r_eig(rho_m(vect, rho_2_arrays))

array([6.39649654e-18, 2.27759896e-17, 2.27759896e-17, 2.97340303e-17,
       1.05847838e-08, 1.05847839e-08, 8.12469946e-02, 8.12469946e-02,
       8.12469946e-02, 8.12469946e-02, 8.12469946e-02, 8.12469946e-02,
       8.26670825e-02, 8.26670825e-02, 8.26670825e-02, 8.26670825e-02,
       8.26670825e-02, 8.26670825e-02, 3.36085923e-01, 3.36085923e-01,
       3.36085923e-01, 3.36085923e-01, 3.36085923e-01, 3.36085923e-01,
       7.48001237e-01, 7.48001237e-01, 7.51998753e-01, 7.51998753e-01])

#### Testing

In [256]:
vect = [-4.646928018967011e-01,2.770294629368069e-13,6.042253100720720e-13,-6.024516357133285e-13,-2.567187786718633e-13,1.437594192304828e-01,-2.763730452330614e-13,4.646928018967681e-01,6.783135520862164e-13,6.755585098225312e-13,-1.437594192304206e-01,2.565069368880155e-13,-6.036534951757788e-13,-6.777043000828106e-13,-6.255919748617283e-14,-2.875188384604817e-01,-6.292320178844996e-13,-5.606330583371935e-13,6.014298958703158e-13,-6.759194003610720e-13,-2.875188384604817e-01,-6.207393804663961e-14,-6.269833844031960e-13,5.590088393571927e-13,2.557458444682729e-13,-1.437594192304207e-01,6.287588334290853e-13,6.270036860946324e-13,4.000749631766127e-01,-2.383049928519640e-13,1.437594192304827e-01,-2.559215089660516e-13,5.600763473792744e-13,-5.580405389843103e-13,2.382296361695720e-13,-4.000749631765536e-01]
vect = np.array(vect)
vect

array([-4.64692802e-01,  2.77029463e-13,  6.04225310e-13, -6.02451636e-13,
       -2.56718779e-13,  1.43759419e-01, -2.76373045e-13,  4.64692802e-01,
        6.78313552e-13,  6.75558510e-13, -1.43759419e-01,  2.56506937e-13,
       -6.03653495e-13, -6.77704300e-13, -6.25591975e-14, -2.87518838e-01,
       -6.29232018e-13, -5.60633058e-13,  6.01429896e-13, -6.75919400e-13,
       -2.87518838e-01, -6.20739380e-14, -6.26983384e-13,  5.59008839e-13,
        2.55745844e-13, -1.43759419e-01,  6.28758833e-13,  6.27003686e-13,
        4.00074963e-01, -2.38304993e-13,  1.43759419e-01, -2.55921509e-13,
        5.60076347e-13, -5.58040539e-13,  2.38229636e-13, -4.00074963e-01])

In [167]:
np.sort(np.linalg.eigvals(rho_m(vect, rho_2_arrays).todense()))

array([6.39649654e-18+0.00000000e+00j, 2.27759896e-17-1.60872919e-17j,
       2.27759896e-17+1.60872919e-17j, 2.97340303e-17+0.00000000e+00j,
       1.05847838e-08+0.00000000e+00j, 1.05847839e-08+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       7.48001237e-01+0.00000000e+00j, 7.48001237e-01+0.00000000e+00j,
       7.51998753e-01+0.00000000e+00j, 7.51998753e-01+0.00000000e+00j])